<a href="https://colab.research.google.com/github/lugabiel/sistemas-nebulosos/blob/desenvolvimento/Solu%C3%A7%C3%A3o_Aproximacao_de_Funcao_Univaria%CC%81vel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

Neste exercício vocês irão utilizar o conceito de conjuntos nebulosos para fazer aproximação de funções univariáveis através de composições de aproximações lineares locais.

Inicialmente será apresentado um exemplo, passo a passo, de como usar conjuntos nebulosos para fazer a aproximação da função $y=x^2$ para $x \in [-1,1]$.

Em seguida, vocês irão realizar a aproximação da função:

$$
 y= \frac{\sin(x)}{x}
$$

para $x \in [0.001, \pi]$

Inicialmente iremos gerar o conjunto de amostras de para aproximação numérica. Iremos utilizar $n=100$ amostras para realizar aproximação numérica da função $y=x^2$

In [31]:
n = 100 # Numero de amostras 

x= np.linspace(-1,1,n)
y = x**2
px.line(x=x,y=y)

Esta função pode ser aproximada pelas seguintes regras:


- Se x é Negativo Então y = -x
- Se x é Positivo Então y = x

In [32]:
x = np.linspace(-1,1,100)
y = x**2


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    line_color='blue',
                    name=r'y'))
fig.add_trace(go.Scatter(x=x, y=-x,
                    mode='lines',
                    line_color='cyan',
                    name=r'$-x$'))
fig.add_trace(go.Scatter(x=x, y=x,
                    mode='lines',
                    line_color='green',
                    name=r'$x$'))
fig.update_layout(yaxis_range=[0,1])
fig.show()

Assumindo que os conjuntos *Negativo* e *Positivo* são conjuntos ordinários, a saída do modelo será definida como:

$$ \hat{y}^{(i)}=
\begin{cases}
-x^{(i)}, \,\ x^{(i)} \leq 0\\
x^{(i)}, \,\ x^{(i)}  \gt 0
\end{cases}
$$
para $i \in [1 \cdots n]$

In [33]:
y_hat = -x
y_hat[int(n/2):] = x[int(n/2):]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    line_color='blue',
                    name=r'y'))
fig.add_trace(go.Scatter(x=x, y=y_hat,
                    mode='lines',
                    line_color='red',
                    name=r'$\hat{y}$'))
fig.update_layout(yaxis_range=[0,1])
fig.show()

O erro de aproximação ($y-\hat{y}$) é ilustrado na figura abaixo.

In [34]:
res = y-y_hat
px.line(x=x, y=res, labels={'y': 'Erro'})

Podemos sumarizar o erro de aproximação utilizando uma métrica de desempenho, como o *Root Mean Squared Error (RMSE)*:

\begin{equation}
 RMSE = \sqrt{\frac{\sum_{i=1}^n (y^{(i)}-\hat{y}^{(i)})^2}{n}}
\end{equation}

In [35]:
rmse = np.sqrt(np.sum((res)**2)/n)
print('RMSE: %.4f'% rmse)

RMSE: 0.1817


Vamos agora, utilizar conjuntos nebulosos para representar os conceitos de números *negativos* e *positivos*.

A figura da próxima célula ilustra as funções de pertinência que definem cada um desses conjuntos. Note que as funções se sobrepõem. Isto é importante para gerar o efeito de suavização da aproximação.

In [36]:
def mu_neg(x):
  return -0.5*x + 0.5

def mu_pos(x):
  return 0.5*x + 0.5

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=mu_neg(x),
                    mode='lines',
                    line_color='black',
                    name=r'$\mu_{neg}(x)$'))
fig.add_trace(go.Scatter(x=x, y=mu_pos(x),
                    mode='lines',
                    line_color='black',
                    name=r'$\mu_{pos}(x)$'))
fig.show()

O modelo é definido pelas seguintes regras nebulosas:

- Se x é Negativo Então $y = y_1$
- Se x é Positivo Então $y = y_2$

Onde os conjuntos *Negativo* e *Positivo* são conjuntos nebulosos com funções de pertinência detalhadas pelo gráfico acima e $y_1 = -x$ e $y_2 = x$.

A saída do modelo é definida como:




\begin{equation}
\hat{y}^{(i)} = \frac{\mu_{neg}(x^{(i)})y_1(x^{(i)}) + \mu_{pos}(x^{(i)})y_2(x^{(i)})}{\mu_{neg}(x^{(i)}) + \mu_{pos}(x^{(i)})}
\end{equation}


In [37]:
y_hat = np.empty([n])
for i in range(n):
  y1 = -x[i]
  y2 = x[i]
  y_hat[i] = mu_neg(x[i])*y1 + mu_pos(x[i])*y2
  y_hat[i] /= mu_neg(x[i])+mu_pos(x[i])

## Versão otimizada do código acima (vetorizada)
#mu_neg_vals = mu_neg(x)
#mu_pos_vals = mu_pos(x)
#y_hat = (mu_neg_vals*-x + mu_pos_vals*x)/(mu_neg_vals + mu_pos_vals)

A figura abaixo ilustra a saída do modelo ($\hat{y}$) e a função a ser aproximada ($y$).

In [38]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    line_color='blue',
                    name=r'y'))
fig.add_trace(go.Scatter(x=x, y=y_hat,
                    mode='lines',
                    line_color='red',
                    name=r'$\hat{y}$'))
fig.update_layout(yaxis_range=[0,1])
fig.show()

O erro de aproximação é ilustrado na figura abaixo.

In [39]:
res = y-y_hat
px.line(x=x, y=res, labels={'y': 'Erro'})

A seguir calcula-se o RMSE.

In [40]:
rmse = np.sqrt(np.sum((res)**2))
print('RMSE: %.4f'% rmse)

RMSE: 0.0000


Agora vocês devem utilizar o mesmo processo para aproximar a função:

$$
 y= \frac{\sin(x)}{x}
$$

no intervalo $x \in [0.001, \pi]$

Os conjuntos nebulosos dos antecedentes das regras devem ter função de pertinência gaussiana.

Iremos utilizar $n=500$ amostras para realizar a aproximação numérica.

In [41]:
n = 500 # Numero de amostras 

x = np.linspace(0.001,np.pi,n)
#x = np.linspace(-np.pi,np.pi,n)
y = np.sinc(x)

px.line(x=x,y=y)

Sugestões:

- Utilizem no mínimo 3 regras
- Definam os valors de $c$ de cada função de pertinência de forma posicioná-las em valores de x em que a aproximação linear local da função seja bem definida
- Definam os valores de $\sigma$ de forma a garantir uma sobreposição entre funções de pertinência adjacentes
- Para definir a reta associada a cada regra, estimem uma reta que seja uma boa aproximação local da função $y$ na região de $x$ da função de pertinência do conjunto definido no antecedente da regra

Realizem os mesmos passos do exemplo anterior (apenas para a aproximação baseada em regras nebulosas) e apresentem no final, um gráfico com a aproximação, um gráfico do erro de aproximação e o RMSE.

A saído do modelo para mais de 2 regras é definida como:

$$
\hat{y}^{(i)} = \frac{\sum_{j=1}^{nr} \mu_j(x^{(i)}) y_j(x^{(i)})}{\sum_{j=1}^{nr} \mu_j(x^{(i)})}
$$

onde $nr$ é o número de regras do modelo.

---

<h2> Solução</h2> 
Para obter a aproximação decidiu-se por utilizar 3 regras diferentes 
para o modelo, complexidade minima requerida no problema proposto.
Cada uma está centrada nos pontos onde a derivada da função objetivo 
sinc(x) [0,pi]; 

In [42]:
#regra1
y_quase = - x**2 + 1 
#regra2
y_quase2 = +(x)**2 -2.84*x + 1.80
#regra3
y_quase3 = -(x)**2 +2.46*2*x - 5.922

<h2> Funções base - Gráfico</h2> 

In [43]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    line_color='blue',
                    name=r'y'))
fig.add_trace(go.Scatter(x=x, y=y_quase,
                    mode='lines',
                    line_color='red',
                    name=r'$\hat{y}$'))
fig.add_trace(go.Scatter(x=x, y=y_quase2,
                    mode='lines',
                    line_color='yellow',
                    name=r'$\hat{y}$'))
fig.add_trace(go.Scatter(x=x, y=y_quase3,
                    mode='lines',
                    line_color='coral',
                    name=r'$\hat{y}$'))
fig.update_layout(yaxis_range=[0,1])
fig.show()

<h2> Regras nebulosas - Gráfico </h2> 

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=mu_q,
                    mode='lines',
                    line_color='fuchsia',
                    name=r'$\mu_{q1}(x)$'))
fig.add_trace(go.Scatter(x=x, y=mu_q2,
                    mode='lines',
                    line_color='red',
                    name=r'$\mu_{q2}(x)$'))
fig.add_trace(go.Scatter(x=x, y=mu_q3,
                    mode='lines',
                    line_color='orange',
                    name=r'$\mu_{q3}(x)$'))
fig.show()

In [44]:
def quase(x):
  return - x**2 + 1
def quase2(x):
  return +(x)**2 -2.84*x + 1.80
def quase3(x):
  return - (x)**2 +2.46*2*x - 5.922

def mu_quase(x):
  if x < 0 or x > 1.42:
    return 0.001 
  if x >= 0:
    return -x + 1.421
  if x < 1.42:
    return -x + 1.421
def mu_quase2(x):
  if x > 2.49 or x <0:
    return 0.001
  if x <= 1.42:
    return x*0.70 + 0.001
  if x > 1.42:
    return -x*0.8 +2.12
def mu_quase3(x):
  if x > np.pi or x < 1.42:
    return 0.001
  if x < 2.49 :
    return x - 1.52
  if x > 2.49:
    return -x + 3.4



O modelo é definido pelas seguintes regras nebulosas, sendo 'x=0', 'x = 1.42' e 'x = 2.49' os pontos onde a derivada da função objetivo é igual a zero:

- Se x > 0 e x < 1.42    --- Então $y = y.quase$
- Se x > 0.71 e x < 2.45 --- Então $y = y.quase2$
- Se x > 1.42            --- Então $y = y.quase3$


Onde os conjuntos *quase*, *quase2*, *quase3* são conjuntos nebulosos que são aproximações locais com funções de pertinência detalhadas pelos gráficos de:

* mu_quase(x)  

- mu_quase2(x) 

- mu_quase3(x)

A saída do modelo é definida por:




\begin{equation}
\hat{y}^{(i)} = \frac{\mu_{mu.q1}(x^{(i)})q1(x^{(i)}) +\mu_{mu.q2}(x^{(i)})q2(x^{(i)}) +\mu_{mu.q3}(x^{(i)})q3(x^{(i)}) + }{\mu_{mu.q1}(x^{(i)}) + \mu_{mu.q2}(x^{(i)}) + \mu_{mu.q3}(x^{(i)})} 
\end{equation}


In [52]:
y_hat = np.empty([n])
mu_q, mu_q2, mu_q3 = np.empty([n]),np.empty([n]),np.empty([n])
for i in range(n):
  y1 = quase(x[i])
  y2 = quase2(x[i])
  y3 = quase3(x[i])

  y_hat[i] = mu_quase(x[i])*y1  + mu_quase2(x[i])*y2 + mu_quase3(x[i])*y3
  y_hat[i] /= mu_quase(x[i]) + mu_quase2(x[i]) + mu_quase3(x[i])


In [53]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y_hat,
                    mode='lines',
                    line_color='black',
                    name=r'$\mu_{neg}(x)$'))
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    line_color='blue',
                    name=r'$\mu_{neg}(x)$'))

fig.show()

O erro de aproximação é ilustrado na figura abaixo.

In [48]:
res = y-y_hat

px.line(x=x, y=(res), labels={'y': 'Erro'})


A seguir calcula-se o RMSE (root mean square error ou raiz do erro médio quadrático).

In [54]:
rmse = np.sqrt(np.sum((res)**2))
print('RMSE: %.4f'% rmse)

RMSE: 1.7958
